# Purpose

This code aims at creating a (csv-file) which connects HR-Files and GE-Files of the same survey to each other.
The final file will store for each survey the HR-File name, the GE-file name, the year (derived from the GE-file) and the country abbreviation.

This code assumes that the program ***creating_water_source_files*** was already executed successfully.Hence, all HR-Files which do not have a corresponding GE-File are saved in a seperate folder.


In [20]:
import os
import pandas as pd
import numpy as np

In [34]:
def check_year_country (gps_dir, water_file):
    dir_gps = os.listdir(gps_dir)
     
    for gps_csv in dir_gps:
        if gps_csv.endswith('.csv'):
            gps_file = os.path.join(gps_dir, gps_csv)
            gps_years = pd.read_csv(gps_file, usecols = ['year'])
            gps_year = gps_years.iloc[1]['year']
            water_years = pd.read_csv(water_file, usecols = ['year'])
            #if gps_year['year'].equals(water_year['year']):
            if any(water_years.iloc[i]['year'] == gps_year for i in range(len(water_years))):
                if len(gps_years) == len(water_years):
                    country_gps = os.path.basename(gps_file)[:2]
                    country_water = os.path.basename(water_file)[:2]
                    if country_gps == country_water:
                        #water = os.path.basename(water_file[:water_file.rfind('-')])
                        gps = os.path.basename(gps_file[:gps_file.rfind(".")])
                        return gps
    return None

def ge_year_country(ge_dir, ge_name):
    
    ge_file = os.path.join(ge_dir, ge_name+'.csv')
    ge_years = pd.read_csv(ge_file, usecols = ['year'])
    ge_year = ge_years.iloc[1]['year']
    ge_country = os.path.basename(ge_name)[:2]
    
    return int(ge_year), ge_country

def create_correspondes_file(hr_dir_,ge_dir):
    
    hr_files = os.listdir(hr_dir)
    ge_files = os.listdir(ge_dir)
    corr_columns = ['HR', 'GE', 'country', 'year']
    corr_list = []
    
    for hr_file in hr_files:
        if hr_file.endswith('.csv'):
            hr_name = hr_file[:hr_file.rfind('-')]
            possible_gps_name = hr_name.replace('HR', 'GE', 1)
            #Check if name is identical if HR is replaced with GE
            if any(possible_gps_name in ge_file for ge_file in ge_files):
                ge_name = possible_gps_name
            else:
                hr_file_path = os.path.join(hr_dir, hr_file)
                ge_name = check_year_country(ge_dir, hr_file_path)
            
            if ge_name is not None:
                year, country = ge_year_country(ge_dir, ge_name)
                survey_list = [hr_name, ge_name, country, year]
                corr_list.append(survey_list)
    
    #Turn list into pandas data frame with corresponding columns and save it as csv
    corr_array = np.array(corr_list)
    corr_df = pd.DataFrame(corr_array, columns = corr_columns)
            
    return(corr_df)

In [35]:
#Main
hr_dir = '/home/shannon/Dokumente/Dokumente/studium/ASA/Projekt/SatelliteImage__GEE/correlation/SAV_Data/water-source'
ge_dir = '/home/shannon/Dokumente/Dokumente/studium/ASA/Projekt/SatelliteImage__GEE/correlation/GPS_Data/gps_csv'
#Path we want to save the correspondence file to
corr_path = os.path.join('/home/shannon/Dokumente/Dokumente/studium/ASA/Projekt/NN/sentinel', 'corresponding_ge_hr_survey.csv')

corr_df = create_correspondes_file(hr_dir,ge_dir)

corr_df.to_csv(corr_path,index = False)
corr_df

,HR,GE,country,year
0,AOHR71FL,AOGE71FL,AO,2015
1,BFHR7AFL,BFGE7AFL,BF,2017
2,BFHR71FL,BFGE71FL,BF,2014
3,BJHR71FL,BJGE71FL,BJ,2017
4,BUHR71FL,BUGE71FL,BU,2016
5,CDHR61FL,CDGE61FL,CD,2013
6,CMHR71FL,CMGE71FL,CM,2018
7,ETHR71FL,ETGE71FL,ET,2016
8,ETHR81FL,ETGE81FL,ET,2019
9,GHHR7BFL,GHGE7AFL,GH,2016
